In [4]:
# import neccessary dependencies
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score

# Import the data

In [5]:
#training data
train = pd.read_csv(r"C:\Users\Admin\OneDrive\Documents\Webdev\hatespeech\train.csv", nrows=5000)

#testing data
test = pd.read_csv(r"C:\Users\Admin\OneDrive\Documents\Webdev\hatespeech\test.csv", nrows=5000)


# # #training data
# train = pd.read_csv(r"C:\Users\jmest\Documents\Files\C4S2\Thesis\ThesisFiles-main\ThesisFiles-main\hatespeech\train.csv", nrows=5000)

# # #testing data
# test = pd.read_csv(r"C:\Users\jmest\Documents\Files\C4S2\Thesis\ThesisFiles-main\ThesisFiles-main\hatespeech\test.csv", nrows=5000)




# Clean the data of null values

In [6]:
# drop rows with null values
train.dropna()

,text,label
0,Inaasahan na ni Vice President Jejomar Binay n...,0
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1
2,Salamat sa walang sawang suporta ng mga taga m...,0
3,@rapplerdotcom putangina mo binay TAKBO PA,1
4,"Binay with selective amnesia, forgetting about...",0
...,...,...
4995,Awww DUTERTE Na wag Lang si Roxas,0
4996,RT @mikkieugenio: If the SC disqualifies Poe a...,0
4997,"""Pag naging presidente si Binay, wala kayong t...",0
4998,Yan na naman ang walang kwentang commercial ni...,1


In [7]:
#check for null values in train
train.isnull().sum()

text     0
label    0
dtype: int64

In [8]:
# check for 0 values in train
sum(train["label"] == 0)

2653

In [9]:
# check for 1 values in train
sum(train["label"] == 1)

2347

# Clean the data of unwated Text and Characters

In [10]:
# remove special characters using the regular expression library

import re

#set up punctuations we want to be replaced

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [11]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [12]:
# clean training data
train_tweet = clean_tweets(train["text"])
train_tweet = pd.DataFrame(train_tweet)

In [13]:
# append cleaned tweets to the training data
train["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
train.head(10)

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya n...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about ...
5,It doesn't matter whoever won between Duterte ...,0,it doesnt matter whoever won between duterte &...
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1,nognog pero nognog din ang nag malasakit wtf t...
7,#OnlyB1nay ?? #FB https://t.co/QEQnsK67Gm,1,
8,What Abi Binay said on running for Makati mayo...,0,what abi binay said on running for makati mayor
9,Srsly. How can Binay do away with no tax for t...,1,srsly how can binay do away with no tax for th...


In [14]:
sum(train['clean_tweet'] == '')

253

In [15]:
#df['Another'] = df['Another'].replace('', np.nan)
#replace all empty spaces with NaN to drop using dropna
train['clean_tweet'] = train['clean_tweet'].replace('', np.NaN)

In [17]:
train.dropna(axis='rows').head(10)

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya n...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about ...
5,It doesn't matter whoever won between Duterte ...,0,it doesnt matter whoever won between duterte &...
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1,nognog pero nognog din ang nag malasakit wtf t...
8,What Abi Binay said on running for Makati mayo...,0,what abi binay said on running for makati mayor
9,Srsly. How can Binay do away with no tax for t...,1,srsly how can binay do away with no tax for th...
10,"Sen Grace Poe, puro ka puso. Kaya lugmok bansa...",1,sen grace poe puro ka puso kaya lugmok bansa n...


In [ ]:
#https://statisticsglobe.com/drop-rows-blank-values-from-pandas-dataframe-python
train['clean_tweet'] = train['clean_tweet'].replace('', float('NaN'), regex = True)

In [ ]:
train.dropna(inplace= True)
train = train.reset_index(drop=True)

In [ ]:
first_column = train.pop('label')
train.insert(0,'label',first_column)

In [ ]:
train.head(10)

,label,text,clean_tweet
0,0,Inaasahan na ni Vice President Jejomar Binay n...,inaasahan na ni vice president jejomar binay n...
1,1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,mar roxas tang ina tuwid na daan daw eh sya n...
2,0,Salamat sa walang sawang suporta ng mga taga m...,salamat sa walang sawang suporta ng mga taga m...
3,1,@rapplerdotcom putangina mo binay TAKBO PA,putangina mo binay takbo pa
4,0,"Binay with selective amnesia, forgetting about...",binay with selective amnesia forgetting about ...
5,0,It doesn't matter whoever won between Duterte ...,it doesnt matter whoever won between duterte &...
6,1,Nognog? Pero nognog din ang nag malasakit? Wtf...,nognog pero nognog din ang nag malasakit wtf t...
7,0,What Abi Binay said on running for Makati mayo...,what abi binay said on running for makati mayor
8,1,Srsly. How can Binay do away with no tax for t...,srsly how can binay do away with no tax for th...
9,1,"Sen Grace Poe, puro ka puso. Kaya lugmok bansa...",sen grace poe puro ka puso kaya lugmok bansa n...


In [ ]:
#total data entries for training

print(train.shape)

(4747, 3)


In [ ]:
#remove stopwords
import urllib.request, json 
with urllib.request.urlopen("https://raw.githubusercontent.com/stopwords-iso/stopwords-tl/master/stopwords-tl.json") as url:
    stopwords = json.loads(url.read().decode())
    print(stopwords)

['akin', 'aking', 'ako', 'alin', 'am', 'amin', 'aming', 'ang', 'ano', 'anumang', 'apat', 'at', 'atin', 'ating', 'ay', 'bababa', 'bago', 'bakit', 'bawat', 'bilang', 'dahil', 'dalawa', 'dapat', 'din', 'dito', 'doon', 'gagawin', 'gayunman', 'ginagawa', 'ginawa', 'ginawang', 'gumawa', 'gusto', 'habang', 'hanggang', 'hindi', 'huwag', 'iba', 'ibaba', 'ibabaw', 'ibig', 'ikaw', 'ilagay', 'ilalim', 'ilan', 'inyong', 'isa', 'isang', 'itaas', 'ito', 'iyo', 'iyon', 'iyong', 'ka', 'kahit', 'kailangan', 'kailanman', 'kami', 'kanila', 'kanilang', 'kanino', 'kanya', 'kanyang', 'kapag', 'kapwa', 'karamihan', 'katiyakan', 'katulad', 'kaya', 'kaysa', 'ko', 'kong', 'kulang', 'kumuha', 'kung', 'laban', 'lahat', 'lamang', 'likod', 'lima', 'maaari', 'maaaring', 'maging', 'mahusay', 'makita', 'marami', 'marapat', 'masyado', 'may', 'mayroon', 'mga', 'minsan', 'mismo', 'mula', 'muli', 'na', 'nabanggit', 'naging', 'nagkaroon', 'nais', 'nakita', 'namin', 'napaka', 'narito', 'nasaan', 'ng', 'ngayon', 'ni', 'nila',

In [ ]:
#data['content2'] =data['Content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
train['rm_stpwrds'] = train['clean_tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords) ]))

In [ ]:
train[['clean_tweet', 'rm_stpwrds']].head(10)

,clean_tweet,rm_stpwrds
0,inaasahan na ni vice president jejomar binay n...,inaasahan vice president jejomar binay taong
1,mar roxas tang ina tuwid na daan daw eh sya n...,mar roxas tang ina tuwid daan daw eh sya nga d...
2,salamat sa walang sawang suporta ng mga taga m...,salamat sawang suporta taga makati pagbabalik ...
3,putangina mo binay takbo pa,putangina mo binay takbo
4,binay with selective amnesia forgetting about ...,binay with selective amnesia forgetting about ...
5,it doesnt matter whoever won between duterte &...,it doesnt matter whoever won between duterte &...
6,nognog pero nognog din ang nag malasakit wtf t...,nognog nognog nag malasakit wtf tangina mo bin...
7,what abi binay said on running for makati mayor,what abi binay said on running for makati mayor
8,srsly how can binay do away with no tax for th...,srsly how can binay do away with no tax for th...
9,sen grace poe puro ka puso kaya lugmok bansa n...,sen grace poe puro puso lugmok bansa natin kah...


In [ ]:
#tokenization 

import nltk
from nltk.tokenize import WhitespaceTokenizer
train['tokenize'] = train['rm_stpwrds'].apply(nltk.tokenize.WhitespaceTokenizer().tokenize) 
train['tokenize'].head()



0    [inaasahan, vice, president, jejomar, binay, t...
1    [mar, roxas, tang, ina, tuwid, daan, daw, eh, ...
2    [salamat, sawang, suporta, taga, makati, pagba...
3                        [putangina, mo, binay, takbo]
4    [binay, with, selective, amnesia, forgetting, ...
Name: tokenize, dtype: object

In [ ]:
train[['rm_stpwrds','tokenize']].head(10)

,rm_stpwrds,tokenize
0,inaasahan vice president jejomar binay taong,"[inaasahan, vice, president, jejomar, binay, t..."
1,mar roxas tang ina tuwid daan daw eh sya nga d...,"[mar, roxas, tang, ina, tuwid, daan, daw, eh, ..."
2,salamat sawang suporta taga makati pagbabalik ...,"[salamat, sawang, suporta, taga, makati, pagba..."
3,putangina mo binay takbo,"[putangina, mo, binay, takbo]"
4,binay with selective amnesia forgetting about ...,"[binay, with, selective, amnesia, forgetting, ..."
5,it doesnt matter whoever won between duterte &...,"[it, doesnt, matter, whoever, won, between, du..."
6,nognog nognog nag malasakit wtf tangina mo bin...,"[nognog, nognog, nag, malasakit, wtf, tangina,..."
7,what abi binay said on running for makati mayor,"[what, abi, binay, said, on, running, for, mak..."
8,srsly how can binay do away with no tax for th...,"[srsly, how, can, binay, do, away, with, no, t..."
9,sen grace poe puro puso lugmok bansa natin kah...,"[sen, grace, poe, puro, puso, lugmok, bansa, n..."


In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
def lema_words(text):
  wnl=WordNetLemmatizer()
  return[wnl.lemmatize(w) for w in text]

train['lematize']=train['tokenize'].apply(lema_words)  
train[['tokenize','lematize']].sample(5)

,tokenize,lematize
2446,"[nya, demenem, i, binay, a, nyaaa, huhu]","[nya, demenem, i, binay, a, nyaaa, huhu]"
4191,"[when, i, woke, up, i, was, shocked, that, so,...","[when, i, woke, up, i, wa, shocked, that, so, ..."
437,"[binay, says, he, doesnt, know, where, friend,...","[binay, say, he, doesnt, know, where, friend, ..."
2941,"[seryoso, tacloban, nga, napala, kay, mar, rox...","[seryoso, tacloban, nga, napala, kay, mar, rox..."
672,"[hay, nako, poe, di, mo, talaga, kayang, bangg...","[hay, nako, poe, di, mo, talaga, kayang, bangg..."


# Proceeding to Training

In [ ]:
# Train the SVM model
clf = svm.SVC()
clf.fit(X_train, y_train)

# Test the model on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

NameError: name 'X_train' is not defined

In [ ]:
#training with lemmatized words
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train['lematize'], train['label'], test_size=0.3, random_state=1)


In [ ]:
# Vectorize the sentences using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(map(' '.join, X_train))
X_test = vectorizer.transform(map(' '.join, X_test))

In [ ]:
# Train the SVM model
clf = svm.SVC()
clf.fit(X_train, y_train)

# Test the model on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7242105263157895


In [ ]:
PREFIX_SET = [
    'nakikipag', 'pakikipag',
    'pinakama', 'pagpapa',
    'pinagka', 'panganga',
    'makapag', 'nakapag',
    'tagapag', 'makipag',
    'nakipag', 'tigapag',
    'pakiki', 'magpa',
    'napaka', 'pinaka',
    'ipinag', 'pagka',
    'pinag', 'mapag',
    'mapa', 'taga',
    'ipag', 'tiga',
    'pala', 'pina',
    'pang', 'naka',
    'nang', 'mang',
    'sing',
    'ipa', 'pam',
    'pan', 'pag',
    'tag', 'mai',
    'mag', 'nam',
    'nag', 'man',
    'may', 'ma',
    'na', 'ni',
    'pa', 'ka',
    'um', 'in',
    'i',
]

INFIX_SET = [
    'um', 'in',
]

SUFFIX_SET = [
    'syon', 'dor',
    'ita', 'han',
    'hin', 'ing',
    'ang', 'ng',
    'an', 'in',
    'g',
]

def tag_lemmatize(word):
    prefix = ''
    suffix = ''
    infix = ''
    root = word

    # Handle repeated prefixes
    for p in PREFIX_SET:
        if word.startswith(p) and word[len(p):].startswith(p):
            prefix = p
            root = word[len(p):]
            break
        elif word.startswith(p):
            prefix = p
            root = word[len(p):]
            for p2 in PREFIX_SET:
                if root.startswith(p2):
                    root = root[len(p2):]
                    break
            break

    # Handle repeated suffixes
    for s in SUFFIX_SET:
        if word.endswith(s) and word[-len(s)-len(word[:-len(s)].rstrip(s)):-len(s)] == s:
            suffix = s
            root = word[:-len(s)]
            break
        elif word.endswith(s):
            suffix = s
            root = word[:-len(s)]
            for s2 in SUFFIX_SET:
                if root.endswith(s2):
                    root = root[:-len(s2)]
                    break
            break

    for i in INFIX_SET:
        if i in root:
            infix = i
            root = root.replace(i, '')
            break

    # Handle repeated letters
    root_len = len(root)
    i = 0
    while i < root_len - 1:
        if root[i] == root[i+1]:
            root = root[:i] + root[i+1:]
            root_len -= 1
        else:
            i += 1

    return root


In [ ]:
word = 'naglalaro'
lemmatized_word = tag_lemmatize(word)
print(lemmatized_word) 


naglalaro


In [ ]:
#TAGALOG STEMMER

PREFIX_SET = [
    'nakikipag', 'pakikipag',
    'pinakama', 'pagpapa',
    'pinagka', 'panganga',
    'makapag', 'nakapag',
    'tagapag', 'makipag',
    'nakipag', 'tigapag',
    'pakiki', 'magpa',
    'napaka', 'pinaka',
    'ipinag', 'pagka',
    'pinag', 'mapag',
    'mapa', 'taga',
    'ipag', 'tiga',
    'pala', 'pina',
    'pang', 'naka',
    'nang', 'mang',
    'sing',
    'ipa', 'pam',
    'pan', 'pag',
    'tag', 'mai',
    'mag', 'nam',
    'nag', 'man',
    'may', 'ma',
    'na', 'ni',
    'pa', 'ka',
    'um', 'in',
    'i',
]

INFIX_SET = [
    'um', 'in',
]

SUFFIX_SET = [
    'syon', 'dor',
    'ita', 'han',
    'hin', 'ing',
    'ang', 'ng',
    'an', 'in',
    'g',
]

def tagalog_stemmer(token):
    # remove prefixes
    for prefix in PREFIX_SET:
        if token.startswith(prefix):
            token = token[len(prefix):]
            break
    # remove infixes
    for infix in INFIX_SET:
        if infix in token:
            token = token.replace(infix, '')
            break
    # remove suffixes
    for suffix in SUFFIX_SET:
        if token.endswith(suffix):
            token = token[:-len(suffix)]
            break
    return token


In [ ]:
#ENGLISH STEMMER

from nltk.stem import WordNetLemmatizer

def english_lemmatizer(token):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(token)



In [ ]:
from langdetect import detect
import pandas as pd

lemmatized_tokens = []
for row in train['tokenize']:
    lemmatized_row = []
    for token in row:
        try: 
            language = detect(token)
        except:
            continue
        if language == 'tl':
            lemmatized_token = tagalog_stemmer(token)
        elif language == 'en':
            lemmatized_token = english_lemmatizer(token)
        else:
            lemmatized_token = token
        lemmatized_row.append(lemmatized_token)
    lemmatized_tokens.append(' '.join(lemmatized_row))

train['lemmatize_modstem'] = pd.Series(lemmatized_tokens)


In [ ]:
train[['label','tokenize','lemmatize_modstem']].head(5)

,label,tokenize,lemmatize_modstem
0,0,"[inaasahan, vice, president, jejomar, binay, t...",inaasahan vice president jejomar bay tao
1,1,"[mar, roxas, tang, ina, tuwid, daan, daw, eh, ...",mar roxas t ina tuwid daan daw eh sya nga di s...
2,0,"[salamat, sawang, suporta, taga, makati, pagba...",salamat saw suporta makati babalik bay in makati
3,1,"[putangina, mo, binay, takbo]",putanga mo bay takbo
4,0,"[binay, with, selective, amnesia, forgetting, ...",bay with selective amnesia forgetting about th...


In [ ]:
#tokenization 

import nltk
from nltk.tokenize import WhitespaceTokenizer
train['lemmatize_modstem'] = train['lemmatize_modstem'].apply(nltk.tokenize.WhitespaceTokenizer().tokenize) 
train['lemmatize_modstem'].head(10)

0      [inaasahan, vice, president, jejomar, bay, tao]
1    [mar, roxas, t, ina, tuwid, daan, daw, eh, sya...
2    [salamat, saw, suporta, makati, babalik, bay, ...
3                            [putanga, mo, bay, takbo]
4    [bay, with, selective, amnesia, forgetting, ab...
5    [t, doesnt, matter, whoever, won, between, dut...
6    [nognog, nognog, lasakit, wtf, tanga, mo, bay,...
7    [what, abi, bay, said, on, running, for, makat...
8    [srsly, how, can, bay, do, away, with, no, tax...
9    [sen, grace, poe, puro, puso, lugmok, bansa, t...
Name: lemmatize_modstem, dtype: object

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train['lemmatize_modstem'], train['label'], test_size=0.3, random_state=1)

In [ ]:
# Vectorize the sentences using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(map(' '.join, X_train))
X_test = vectorizer.transform(map(' '.join, X_test))

In [ ]:
# Train the SVM model
clf = svm.SVC()
clf.fit(X_train, y_train)

# Test the model on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7249122807017544
